<a href="https://colab.research.google.com/github/arafat04/bn-hi-MT-improvement-using-llm/blob/main/evaluation_llama8b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import sacrebleu.utils
from sacrebleu.metrics.bleu import BLEU
from tqdm import tqdm
import transformers
import torch

In [ ]:
!nvidia-smi

Sun Feb 16 22:08:10 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A40                     On  |   00000000:81:00.0 Off |                    0 |
|  0%   38C    P8             22W /  300W |       1MiB /  46068MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
from sacrebleu import corpus_bleu
from huggingface_hub import login

In [ ]:
# load the Llama-3.1-8B-Instruct tokenizer
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
#model_id = "meta-llama/Llama-3.2-1B-Instruct"
token = ""  # Replace with your token

# Step 1: Authenticate explicitly
from huggingface_hub import login
login(token=token)

# Step 2: Load with updated settings
tokenizer = AutoTokenizer.from_pretrained(model_id, token=token)

In [ ]:
# Load the model in FP16
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,  # Use FP16
    token=token,
    load_in_8bit=True,  # Optional: Use 8-bit quantization
)

2025-02-16 22:12:06.868910: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739740326.972634  183201 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739740327.028405  183201 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-16 22:12:07.598310: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `Bi

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

### Define the Few-Shot Prompting Function


In [ ]:
def format_input(bengali_text):
    """
    Formats the input for few-shot prompting.
    :param bengali_text: The Bengali text to translate.
    :return: A list of messages for the chat template.
    """
    messages = [
        {"role": "system", "content": "You are an expert on translating from Bengali to Hindi."},
        {"role": "user", "content": "Translate the following Bengali text to Hindi: খবরটা শুনে খুব খারাপ লাগলো।"},
        {"role": "assistant", "content": "खबर सुनकर बहुत दुख हुआ."},
        {"role": "user", "content": "Translate the following Bengali text to Hindi: প্রশিক্ষণ অ্যালগরিদম, পরামিতি, ন্যায্যতা সীমাবদ্বতা বা অন্যান্য প্রয়োগ পদ্ধতি এবং বৈশিষ্ট্য সম্পর্কে তথ্য।"},
        {"role": "assistant", "content": "प्रशिक्षण अल्गोरिदम, पैरामीटर्स, न्यायसंगतता सीमाएँ या अन्य अनुप्रयोग विधियों और विशेषताओं के बारे में जानकारी।"},
        {"role": "user", "content": f"Translate the following Bengali text to Hindi: {bengali_text}"}
    ]
    return messages

In [ ]:
!nvidia-smi

Sun Feb 16 22:15:43 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A40                     On  |   00000000:81:00.0 Off |                    0 |
|  0%   45C    P0             81W /  300W |    8989MiB /  46068MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import sacrebleu

source_file = sacrebleu.utils.get_source_file("wmt21", "bn-hi")
print(f"Data type of source_file: {type(source_file)}")

Data type of source_file: <class 'str'>


In [ ]:
print(f"Source file path: {source_file}")

Source file path: /storage/brno2/home/rahmang/.sacrebleu/wmt21/wmt21.bn-hi.src


### Load the WMT21 Dataset

In [ ]:
with open(source_file, "r", encoding="utf-8") as fd:
    sources = list(map(str.strip, fd.readlines()))

# Print the first 5 lines of the source file
print("First 5 lines of the source file:")
for line in sources[:5]:
    print(line)

First 5 lines of the source file:
ইকুয়েডরীয় গ্যালোপেগোস দ্বীপপুঞ্জের এক নতুন প্রজাতির পাখির গঠন নিয়ে জার্নাল সায়েন্স রিপোর্টে বৃহস্পতিবার একটি গবেষণাপত্র প্রকাশিত হয়েছে।
মার্কিন যুক্তরাষ্ট্রের প্রিন্সটন বিশ্ববিদ্যালয় এবং সুইডেনের ইউপসালা বিশ্ববিদ্যালয়ের গবেষকরা জানিয়েছেন যে নতুন প্রজাতিটি শুধুমাত্র দুটি প্রজন্মের মধ্যে বিবর্তিত হয়েছে, যদিও ডারউইন ফিঞ্চ, জেসোপিজা ফোর্টস এবং ইমিগ্র্যান্ট ক্যাকটাসের ফিঞ্চ, জিওপিজা কনিরোস্ট্রিস মধ্যে প্রজননের কারণে এই প্রক্রিয়াটি আরও বেশি সময় নেয় বলে মনে করা হয়।
"এই বিতর্কটি হারিকেন ক্যাটরিনার সময় ত্রাণ ও পুনর্নির্মাণের উপর ব্যয়কে কেন্দ্র করে বেশি ছড়িয়েছিল; যেটাকে কিছু অর্থ সংশ্লিষ্ট রক্ষণশীলরা হাস্যকরভাবে ""বুশের নিউ অরলিন্স চুক্তি"" বলে অবহিত করেছিল।"
পুনর্নির্মাণের চেষ্টা সম্পর্কে উদার সমালোচনা ওয়াশিংটন অভ্যন্তরীণদের পুনর্নির্মাণ চুক্তিগুলি অনুধাবনের দিকে লক্ষ্যস্থির করেছে।
খেলাটি দুর্দান্ত আবহাওয়ার সাথে সকাল ১০:০০ টায় শুরু হয়েছিল এবং মধ্য সকালের দ্রুত পরিষ্কার হয়ে যাওয়া হালকা বৃষ্টিপাত ছাড়া, এটি সপ্তম রাগবির জন্য একটি উপযুক্ত দিন ছিল।


In [ ]:
# Get the reference files
reference_files = sacrebleu.utils.get_reference_files("wmt21", "bn-hi")
print(f"Data type of reference_files: {type(reference_files)}")
print(f"Reference files: {reference_files}")

# Read and print some values from the first reference file
with open(reference_files[0], "r", encoding="utf-8") as fd:
    references = list(map(str.strip, fd.readlines()))

print("First 5 lines of the first reference file:")
for line in references[:5]:
    print(line)

Data type of reference_files: <class 'list'>
Reference files: ['/storage/brno2/home/rahmang/.sacrebleu/wmt21/wmt21.bn-hi.ref-A']
First 5 lines of the first reference file:
जर्नल साइंस में गुरूवार को प्रकाशित एक अध्ययन ने इक्वाडोरियन गैलापागोस आइलैंड पर एक नई पक्षी प्रजाति के निर्माण की सूचना दी.
संयुक्त राज्य अमेरिका में प्रिंसटन विश्वविद्यालय और स्वीडन में उप्साला विश्वविद्यालय के शोधकर्ताओं ने बताया कि नई प्रजातियाँ सिर्फ दो पीढ़ियों में विकसित हुईं, हालाँकि एक स्थानिक डार्विन फिंच, जियोस्पाइज़ा फोर्टेस और अप्रवासी कैक्टस फिंच, जिओस्पिज़ा कोनिरोस्ट्रिस के बीच प्रजनन के कारण इस प्रक्रिया में काफी समय लगने की बात कही गई थी.
तूफ़ान कैटरीना के मद्देनज़र राहत और पुनर्निर्माण पर चल रहे लंबे विवाद से बहस छिड़ गई थी; जिसमें कुछ राजकोषीय रूढ़िवादियों ने “बुश की न्यू ऑरलियन्स डील” का मज़ाक बनाया है.
पुनर्निर्माण की कोशिश की स्पष्ट आलोचना वॉशिंगटन के कथित घनिष्ट लोगों को पुनर्निर्माण के ठेके दिए जाने पर केंद्रित रही है.
खेल सुबह 10:00 बजे बहुत बढ़िया मौसम के साथ शुरू हुआ और सुबह-सुबह की रिमझिम 

In [ ]:
import sacrebleu

# Get the source and reference files
source_file = sacrebleu.utils.get_source_file("wmt21", "bn-hi")
reference_files = sacrebleu.utils.get_reference_files("wmt21", "bn-hi")

# Load the source and reference texts
with open(source_file, "r", encoding="utf-8") as fd:
    sources = [line.strip() for line in fd.readlines()]

with open(reference_files[0], "r", encoding="utf-8") as fd:
    references = [[line.strip() for line in fd.readlines()]]  # Wrap in a list for sacrebleu

In [ ]:
sources[0]

'ইকুয়েডরীয় গ্যালোপেগোস দ্বীপপুঞ্জের এক নতুন প্রজাতির পাখির গঠন নিয়ে জার্নাল সায়েন্স রিপোর্টে বৃহস্পতিবার একটি গবেষণাপত্র প্রকাশিত হয়েছে।'

In [ ]:
references[0][0]

'जर्नल साइंस में गुरूवार को प्रकाशित एक अध्ययन ने इक्वाडोरियन गैलापागोस आइलैंड पर एक नई पक्षी प्रजाति के निर्माण की सूचना दी.'

### Generate Translations Using the Model

In [ ]:
def generate_translation(model, tokenizer, bengali_text):
    """
    Generates a Hindi translation for the given Bengali text using the model.
    :param model: The loaded model.
    :param tokenizer: The loaded tokenizer.
    :param bengali_text: The Bengali text to translate.
    :return: The generated Hindi translation.
    """
    # Format the input with few-shot examples
    messages = format_input(bengali_text)

    # Apply the chat template
    formatted_input = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # Tokenize the input
    inputs = tokenizer(formatted_input, return_tensors="pt").to(model.device)

    # Generate the translation
    with torch.no_grad():
        outputs = model.generate(
            inputs.input_ids,
            max_new_tokens=512,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=0.8,
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode the generated text
    translation = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only the assistant's response
    translation = translation.split("assistant\n")[-1].strip()
    return translation

# Generate translations for all source texts
translations = []
count = 0
for source in sources:
    translation = generate_translation(model, tokenizer, source)
    translations.append(translation)
    if count <5:
        print(f"Source: {source}\nTranslation: {translation}\n")
    count += 1


Source: ইকুয়েডরীয় গ্যালোপেগোস দ্বীপপুঞ্জের এক নতুন প্রজাতির পাখির গঠন নিয়ে জার্নাল সায়েন্স রিপোর্টে বৃহস্পতিবার একটি গবেষণাপত্র প্রকাশিত হয়েছে।
Translation: इक्वेडोरियाई गैलापागोस द्वीपसमूह में एक नए प्रकार के पक्षी के बारे में जानकारी देने वाला एक शोध पत्र बृहस्पतिवार जर्नल सायंस रिपोर्ट में प्रकाशित हुआ है।

Source: মার্কিন যুক্তরাষ্ট্রের প্রিন্সটন বিশ্ববিদ্যালয় এবং সুইডেনের ইউপসালা বিশ্ববিদ্যালয়ের গবেষকরা জানিয়েছেন যে নতুন প্রজাতিটি শুধুমাত্র দুটি প্রজন্মের মধ্যে বিবর্তিত হয়েছে, যদিও ডারউইন ফিঞ্চ, জেসোপিজা ফোর্টস এবং ইমিগ্র্যান্ট ক্যাকটাসের ফিঞ্চ, জিওপিজা কনিরোস্ট্রিস মধ্যে প্রজননের কারণে এই প্রক্রিয়াটি আরও বেশি সময় নেয় বলে মনে করা হয়।
Translation: अमेरिका के प्रिंसटन विश्वविद्यालय और स्वीडन के यूप्साला विश्वविद्यालय के शोधकर्ताओं ने यह जानकारी दी है कि यह नई प्रजाति दो पीढ़ियों के भीतर विकसित हुई है, हालांकि डार्विन फिंच, जेसोपीजा फोर्ट्स और इमिग्रेंट कैक्टस के फिंच, जियोपीजा कोनीरोस्ट्रिस के बीच प्रजनन के कारण यह प्रक्रिया अधिक समय लेती है माना जाता है।

Source: "এই বিতর্কট

### Compute the BLEU Score

In [ ]:
# Compute the BLEU score
bleu_score = corpus_bleu(translations, references)
print(f"BLEU score: {bleu_score.score}")

BLEU score: 15.120354828137353


### Compute BERTScore

In [ ]:
# !pip install bert-score

In [ ]:
from bert_score import score as bert_score  # Import BERTScore


In [ ]:
import sacrebleu
reference_files = sacrebleu.utils.get_reference_files("wmt21", "bn-hi")

with open(reference_files[0], "r", encoding="utf-8") as fd:
    references = [line.strip() for line in fd.readlines()]  # Flatten the list


In [ ]:
references[0:5]

['जर्नल साइंस में गुरूवार को प्रकाशित एक अध्ययन ने इक्वाडोरियन गैलापागोस आइलैंड पर एक नई पक्षी प्रजाति के निर्माण की सूचना दी.',
 'संयुक्त राज्य अमेरिका में प्रिंसटन विश्वविद्यालय और स्वीडन में उप्साला विश्वविद्यालय के शोधकर्ताओं ने बताया कि नई प्रजातियाँ सिर्फ दो पीढ़ियों में विकसित हुईं, हालाँकि एक स्थानिक डार्विन फिंच, जियोस्पाइज़ा फोर्टेस और अप्रवासी कैक्टस फिंच, जिओस्पिज़ा कोनिरोस्ट्रिस के बीच प्रजनन के कारण इस प्रक्रिया में काफी समय लगने की बात कही गई थी.',
 'तूफ़ान कैटरीना के मद्देनज़र राहत और पुनर्निर्माण पर चल रहे लंबे विवाद से बहस छिड़ गई थी; जिसमें कुछ राजकोषीय रूढ़िवादियों ने “बुश की न्यू ऑरलियन्स डील” का मज़ाक बनाया है.',
 'पुनर्निर्माण की कोशिश की स्पष्ट आलोचना वॉशिंगटन के कथित घनिष्ट लोगों को पुनर्निर्माण के ठेके दिए जाने पर केंद्रित रही है.',
 'खेल सुबह 10:00 बजे बहुत बढ़िया मौसम के साथ शुरू हुआ और सुबह-सुबह की रिमझिम फुहारें, जो जल्दी ही ख़त्म हो गई थीं, के अलावा यह 7वे रग्बी के लिए शानदार दिन था.']

In [ ]:
# Compute BERTScore
P, R, F1 = bert_score(translations, references, lang="hi", verbose=True)

# Print BERTScore results
print(f"BERTScore Precision: {P.mean().item()}")
print(f"BERTScore Recall: {R.mean().item()}")
print(f"BERTScore F1: {F1.mean().item()}")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

calculating scores...
computing bert embedding.


  0%|          | 0/16 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/8 [00:00<?, ?it/s]

done in 2.72 seconds, 184.60 sentences/sec
BERTScore Precision: 0.8293283581733704
BERTScore Recall: 0.8275999426841736
BERTScore F1: 0.8283059597015381
